In [ ]:
import patch_manager as pmang
from reconstructor import Reconstructor
from skimage import io
import numpy as np
recon = Reconstructor()

sperm_path2 = 'C:\\Users\\Blomst\\sperm_0p2_40hz_12t_00068.BTF'
X = io.imread(sperm_path2)


test_image = "c:\\users\\blomst\\groundtruth\\87\\5.png"

#img = io.imread(test_image, as_gray=True)*255
#pmang.show_patch_as_image(img, "lol")

In [ ]:
img = io.imread(test_image, as_gray=True)*255



window_size = 128
patches_pr_layer = 15
total_patches = 4096

res1 = []
res2 = []
for window_size in range(6,50):
    print(window_size)
    
    total_patches = 10*window_size*window_size
    patches_pr_layer = int(total_patches/800 + 10)
    rs = pmang.random_coordinates_all(X,patches_pr_layer,window_size, max=total_patches)
    patches = pmang.get_patches(X, rs, window_size)
    recon.get_pca(patches)
    res1.append([window_size, recon.n_var])
    res2.append([window_size*window_size, recon.n_var])
    #print(total_patches, ",", recon.n_var)

In [ ]:
import matplotlib.pyplot as plt
def plot_points(points,x_label):
    x = [point[0] for point in points]
    y = [point[1] for point in points]
    plt.scatter(x, y)
    plt.xlabel(x_label)
    plt.ylabel('Components for 95% variance')
    plt.show()

plot_points(res1, "patch side length")
plot_points(res2, "patch pixel count")

In [ ]:
recon.get_pca(patches)

In [ ]:
reconstruction = recon.reconstruct_image(img, 300, window_size, 4)
pmang.save_patch_as_image(reconstruction, "recon_test.png")

In [ ]:
def convolution(x, i, j):
    I = len(x)-1
    J = len(x[0])-1
    if  (i==0 and j==0):
        return x[i+1,j]*1/2 + x[i+1,j+1]*1/2
    elif(i == I and j == J):
        return x[i-1,j]*1/2 + x[i-1,j-1]*1/2
    elif(i == I and j == 0):
        return x[i-1,j]*1/2 + x[i-1,j+1]*1/2
    elif(i == 0 and j == J):
        return x[i+1,j]*1/2 + x[i+1,j-1]*1/2
    elif(i==0):
        return x[i+1,j]*1/3 + x[i+1,j-1]*1/3 + x[i+1,j+1]*1/3
    elif(j==0):
        return x[i-1,j]*1/3 + x[i+1,j]*1/3 + x[i-1,j+1]*1/6 + x[i+1,j+1]*1/6
    elif(i == I):
        return x[i-1,j]*1/3 + x[i-1,j+1]*1/3 + x[i-1,j-1]*1/3
    elif(j == J):
        return x[i-1,j]*1/3 + x[i+1,j]*1/3 + x[i+1,j-1]*1/6 + x[i-1,j-1]*1/6
    else:
        return x[i-1,j]*1/6 + x[i+1,j]*1/6 + x[i+1,j-1]*1/6 + x[i-1,j+1]*1/6 + x[i+1,j+1]*1/6 + x[i-1,j-1]*1/6
    

def convolve_nth(x, nth, first):
    y = np.copy(x)
    for i in range(len(y)):
        for j in range(len(y[0])):
            if((i-first)%nth == 0):
                y[i,j] = convolution(y, i, j)
    return y

In [ ]:
recon_file = io.imread("recon_test.png")

In [ ]:


pmang.show_patch_as_image(recon_file, "y")

yes = convolve_nth(recon_file, 4, 0)

pmang.show_patch_as_image(yes, "y")
pmang.save_patch_as_image(yes, "recon_convolution.png")

In [ ]:
def change_range(img):
    new_img = np.copy(img)
    min = 100
    max = 130
    new_max = 255
    for i in range(len(img)):
        for j in range(len(img[0])):
            new_number = (img[i][j]-min)/(max - min ) * new_max
            new_img[i][j] = new_number
    return new_img

pmang.show_patch_as_image(change_range(reconstruction), "yeet")

In [ ]:
patch = X[87,6,20:148,50:178]
patch_full = X[87,6]
removed = pmang.remove_row_from_patch(patch, 34, window_size, recon.mu)
removed = pmang.remove_row_from_patch(removed, 35, window_size, recon.mu)
removed = pmang.remove_row_from_patch(removed, 36, window_size, recon.mu)
pmang.save_patch_as_image(patch, "original.png")
pmang.save_patch_as_image(patch_full, "original_full.png")
pmang.save_patch_as_image(removed, "removed3.png")

path1 = "recon\\"
path2 = "comps_k\\"
path3 = "comps\\"

pmang.make_directory(path1)
pmang.make_directory(path2)
pmang.make_directory(path3)
for i in range(4096):
    vec1 = recon.reconstruct_patch(removed, i)
    vec2 = recon.reconstruct_patch_n1(removed, i)*15 + recon.mu
    img1 = pmang.vec_2_patch(vec1,window_size)
    img2 = pmang.vec_2_patch(vec2,window_size)

    out1 = path1 + str(i) + ".png"
    out2 = path2 + str(i) + ".png"
    pmang.save_patch_as_image(img1,out1) 
    pmang.save_patch_as_image(img2,out2) 

In [ ]:
path3 = "comps\\"
recon.save_principal_components_as_image (15, path3, window_size)


In [ ]:
from PIL import Image
import os

def make_grid(folder_path, output_path):
    images = []
    for i in range(1024):
        filename = str(i) + '.png'
        img = Image.open(os.path.join(folder_path, filename))

        if(i==recon.n_var):
            img = Image.open(os.path.join(folder_path, "0.png"))

        images.append(img)
    width, height = images[0].size
    new_im = Image.new('RGB', (32 * width, 32 * height))
    x_offset = 0
    y_offset = 0
    for i in range(32):
        for j in range(32):
            new_im.paste(images[i*32+j], (x_offset, y_offset))
            x_offset += width
        y_offset += height
        x_offset = 0
    new_im.save(output_path)

make_grid('comps', 'comps.png')
make_grid('comps_k', 'comps_k.png')
make_grid('recon', 'recon.png')

In [ ]:
print(recon.n_var)